# Ewald sum

References:
- "Computer Simulation of Liquids" by Michael P. Allen and Dominic J. Tildesley (2nd edition, Oxford University Press, 2017)
- https://github.com/Allen-Tildesley

In [1]:
import numpy as np
import torch
from scipy.special import erfc

def distance( x ):
    return torch.norm(x[None, :, :] - x[:, None, :], dim=-1)

def qpairs( q ):
    return q[None, :]*q[:, None]

# Real Space

In [2]:
def rwald( x, q, sigma=0.1178511 ):
    """Returns the r-space part of energy of ewald summation
    
    Arguments:
        x (float): position vectors (dim = n x 3)
        q (int): charges (dim = n)
        sigma (float): sqrt(variance), width of gaussian
    
        n (int): number of particles
    
    Output:
        e_short (float): short range part of ewald potential
    """
    epsilon = 8.854187817e-12
    #x = torch.tensor(x, dtype=np.float32)
    #q = torch.tensor(q, dtype=np.float32)
    n, d = list(x.size())
    assert n == q.shape[0], 'dimension error: q needs n entries'
    
    e_short = 0.
    
    r = distance(torch.Tensor(x))
    qiqj = qpairs(torch.Tensor(q))
    r_invers = r.clone()
    r_invers[r_invers!=0] = 1/r_invers[r_invers!=0]
    return torch.sum(
        qiqj * r_invers * erfc( r / (np.sqrt(2) * sigma) )
        ) / (2 * np.pi * epsilon)

In [45]:
charge = torch.tensor([-1., 2, -1, 3])
position = torch.tensor([[0., 0.], [0., 1.], [1., 1.], [1., 0.]])
#qiqj = qpairs(charge)
#print(qiqj)
#dist = distance(position)
#print(dist)
#dist[dist!=0] = 1/dist[dist!=0]
#print(qiqj*dist)
#print(torch.sum(qiqj))
k = torch.tensor([[2., 3], [-1., -3]])
r_ab = position[None, :, :] - position[:, None, :]
#print(k)
#print(r_ab)
#print(k[:, None, None]*r_ab)
#print(torch.sum(k[:, None, None]*r_ab, dim=-1)) 

In [4]:
print(rwald(position, charge))
#a = np.array([[.1, .2, .4], [.8, 1.6, 3.2]])
#print(a)
#print(erfc(a*6))

tensor(-7.7363e-06)


# Fourier space

In [334]:
def nkmax( nmax ):
    mitnull = (nmax + 1) ** 3 * 2 ** 3
    korr = 7 + sum([30 + i * 24 for i in range(nmax)])
    return mitnull - korr

def kvec( maxval=3, ksq=15 ):
    preallo = np.zeros([nkmax(maxval), 3], dtype=np.float32)
    cnt = 0
    for i in range(maxval + 1):
        for j in range(maxval + 1):
            for k in range(maxval + 1):
                if (i**2+j**2+k**2 <= ksq):
                    preallo[cnt] = [i, j, k]
                    cnt += 1
                    if (i > 0):
                        preallo[cnt] = [-i, j, k]
                        cnt += 1
                    if (j > 0):
                        preallo[cnt] = [i, -j, k]
                        cnt += 1
                    if (k > 0):
                        preallo[cnt] = [i, j, -k]
                        cnt += 1
                    if (i > 0 and j > 0):
                        preallo[cnt] = [-i, -j, k]
                        cnt += 1
                    if (i > 0 and k > 0):
                        preallo[cnt] = [-i, j, -k]
                        cnt += 1
                    if (j > 0 and k > 0):
                        preallo[cnt] = [i, -j ,-k]
                        cnt += 1
                    if (i > 0 and j > 0 and k > 0):
                        preallo[cnt] = [-i, -j, -k]
                        cnt += 1
    return torch.from_numpy(preallo[0:cnt])

def dx( x ):
    return x[None, :, :] - x[:, None, :]

def sfactor( x, q, kmax ):
    # check torch.tensor
    # check dimensions: q, k, x
    return torch.sum(
        torch.sum( qpairs( q ) * np.cos( 
        torch.sum( k[:, None, None] * dx( x ), 
                  dim=-1 )), dim=-1), dim=-1)
    

def kwald( x, q, kmax=3, lbox=1 ):
    """Returns the fourier space part of energy of ewald summation
    
    Arguments:
        x (float): position vectors (dim = n x 3)
        q (int): charges (dim = n)
        sigma (float): sqrt(variance), width of gaussian
    
        n (int): number of particles
    
    Output:
        e_l (float): long range part of ewald potential
    """
    sigma = lbox/10.
    epsilon = 8.854187817e-12
    q = charge   
    # x = torch.tensor([[x1, y1, z1], [...], ...])
    # q = torch.tensor([q1, q2, ...])
    k = kvec(kmax,3*kmax**2)*2*np.pi/lbox
    s_sq = sfactor(x, q, k)
    klen = torch.sum(k**2, dim=-1)
    kleninv = klen.clone()
    kleninv[kleninv!=0] = 1/kleninv[kleninv!=0]
    factor = np.exp(-1/2*sigma**2*klen)*kleninv
    return 1/(2*lbox**3*1)*torch.sum(s_sq*factor)
    #return 1/(2 * lbox^3 * epsilon)

In [331]:
#print(kvec(4,3*4**2))
#print(kvec(5,3*5**2))
print(kwald(x, q, 4, 1)) #for i in range(5)])

RuntimeError: The size of tensor a (1331) must match the size of tensor b (729) at non-singleton dimension 0

In [311]:
#print([nkmax(i + 1) for i in range(3)])
k = kvec()
print(len(k))
print(k[0:2])
#indexlist = np.argsort(np.linalg.norm(k,axis=-1))
#print(indexlist)
#print(k[indexlist])

251
tensor([[0., 0., 0.],
        [0., 0., 1.]])


In [333]:
q = charge
x = torch.tensor([[0., 0, 3],
        [0, 1, 3],
        [1, 1, -3],
        [1, 0, -3]])
#print(q)
#print(dx(x))
k = kvec(5)*2*np.pi/1
print(len(k))
#print(k[-3:-1])
#print(sfactor(x, q, k))
inner =  k[:, None, None] * dx( x )
#print(inner)
inner = torch.sum(inner, dim=-1)
#print(inner)
outer = torch.sum( qpairs( q ) * np.cos(inner), dim=-1)
outer = torch.sum(outer, dim=-1)
#print(outer)
klen = torch.sum(k**2, dim=-1)
kleninv = klen.clone()
kleninv[kleninv!=0] = 1/kleninv[kleninv!=0]
factor = np.exp(-1/2*0.1**2*klen)*kleninv
print(1/(2*1**3*1)*torch.sum(outer*factor))

251
tensor(1.8253)
